In [1]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')
sys.path.insert(1, '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src')
print(sys.path)

['/home/jupyter/nl2sql-new/pg-interface', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages']


In [2]:
# Uncomment the below commands if required to re-install the libraries
# %pip install faiss-cpu --quiet
# %pip install google.cloud --quiet
# %pip install cloud-sql-python-connector --quiet
# %pip install pg8000 --quiet
# %pip install langchain-google-vertexai

In [2]:
import numpy as np
import csv
import faiss
from pandas import DataFrame
from datetime import datetime
from tqdm import tqdm

In [3]:
# PROJECT_ID = ! gcloud config get project
# from nl2sql_generic import Nl2sqlBq
PROJECT_ID = 'sl-test-project-363109'
LOCATION = 'us-central1'
DATASET_ID = 'zoominfo'

# Query and Embeddings in PostgreSQL

## Initialise

In [4]:
from nl2sql_query_embeddings import PgSqlEmb

# Make sure you update the following variables in-line with your project
PGPROJ = "sl-test-project-363109"
PGLOCATION = 'us-central1'
PGINSTANCE = "nl2sql-test"
PGDB = "test-db"
PGTABLE = 'documents' #'myqueries'
PGUSER = "postgres"
PGPWD = "nl2sql-test"
# INDX_FILE = 'saved_index_pgdata' # optoinal, considered as default

pge = PgSqlEmb(PGPROJ, PGLOCATION, PGINSTANCE, PGDB, PGUSER, PGPWD, pg_table=PGTABLE)
# mytable=PGTABLE

2024-05-14 18:33:55.015378: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 18:33:57.484413: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-05-14 18:33:57.484857: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

PGSQL - class Initiated


## Create Table

In [9]:
# Create Table in PostgresDB if not already existing
pge.create_table()

## Insert records to PostgreSQL and update index

In [10]:
examples = []
with open('sample_sqls.csv', 'r') as f:
    samples_data = csv.reader(f)
    header = next(samples_data)
    for r in samples_data:
        examples.append(r)

for example in tqdm(examples):
    pge.insert_row(example[0], example[1])

100%|██████████| 19/19 [00:54<00:00,  2.88s/it]


In [12]:
new_question = "How many employees in California are from Livingston County?"
mapped_sql = "select count(*) from `sl-test-project-363109.zoominfo.zi-retail-companies-headquartered-california` where ZI_C_COMPANY_COUNTY = 'Livingston County'"
pge.insert_row(new_question, mapped_sql)

## Retrieve data 

In [5]:
# Retrieve the data from the PostgreDB and return a dataframe
def extract_data():
    tmp = pge.extract_data()
    df = DataFrame(tmp.fetchall())
    print(df)
    return df

df = extract_data()
df
headers = ["question", "sql"]
df.to_csv('sample_sqls.csv', columns=headers, index=False)

ObjectNotExecutableError: Not an executable object: 'SELECT * FROM documents'

## Search the VectorDB for closest matching queries

In [13]:
# Search for the closest matching queries
new_query = 'What is the total revenue for Hospitality industry'
output_json = pge.search_matching_queries(new_query)
output_json

Trying to read the index file ../../nl2sql-generic/nl2sql_src/cache_metadata/saved_index_pgdata


[{'question': ' How do CalWorks program participation trends differ by race and ethnicity?',
  'sql': "SELECT COALESCE(SUM(SAFE_CAST( White   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_whites_1, COALESCE(SUM(SAFE_CAST( Black   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_blacks_1, COALESCE(SUM(SAFE_CAST( Hispanic   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_hispanics_1, COALESCE(SUM(SAFE_CAST( Asian_PI   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_asian_1, COALESCE(SUM(SAFE_CAST( Native_American   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_native_american_1 FROM `cdii-poc.HHS_Program_Counts.calhhs-dashboard-2015-2020-annual-data-file`  WHERE (Program ) = 'CalWorks';"},
 {'question': 'How many of them live in Nevada County?',
  'sql': "SELECT SUM(CAST(Person AS INT64)) AS total_beneficiaries FROM `cdii-poc.HHS_Program_Counts.calhhs-dashboard-2015-2020-annual-data-fil

## Clear VectorDB and re-create VectorDB from all data in PostgreSQL

In [11]:
pge.recreate_vectordb_index()

Number of elements :  14


## Clear all Data in DB

In [12]:
pge.empty_table()
df = extract_data()
df

Empty DataFrame
Columns: []
Index: []


""
